In [0]:
#!import "Storage"

In [0]:
#!import "../Utils/ExtentionMethods"

# Basic Cashflow

In [0]:
public interface ComputeBasicCashflow : IScope<BasicCashflow>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
     builder.ForScope<ComputeBasicCashflow>(s => s
         .WithApplicability<LinearShape>(x => x.Identity.Shape == Shape.Linear)
         .WithApplicability<CompoundShape>(x => x.Identity.Shape == Shape.Compound)
    );
    //Constant is the default Case
    double[] Values => Enumerable.Repeat(Identity.InitialValue, Identity.Length).ToArray();
}

public interface LinearShape : ComputeBasicCashflow {
    double[] ComputeBasicCashflow.Values => Identity.InitialValue.GetLinearValues(Identity.Shift, Identity.Length, Identity.Values);
}

public interface CompoundShape : ComputeBasicCashflow {
    double[] ComputeBasicCashflow.Values => Identity.InitialValue.GetCompoundValues(Identity.Shift, Identity.Length, Identity.Values);
}

In [0]:
public interface BasicCashflows : IScope<CashflowIdentity, ICashflowStorage>
{
    public IEnumerable<(string Layer, double[] Values)> LayerValues => GetStorage().GetCashflow<BasicCashflow>(Identity)
        .Select((cd,i) => ("Basic-" + cd.Shape + i.ToString(), GetScope<ComputeBasicCashflow>(cd).Values));
}

# Bounded Cashflow

In [0]:
public interface ComputeBoundedCashflows : IScope<BoundedCashflow>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
     builder.ForScope<ComputeBoundedCashflows>(s => s
         .WithApplicability<RandomShape>(x => x.Identity.Shape == Shape.Random)
    );
    //null is the default Case
    double[] Values => default;
}

public interface RandomShape: ComputeBoundedCashflows{
    double[] ComputeBoundedCashflows.Values => GetBoundedRandomCashflow(Identity.Shift, Identity.Length, Identity.Maximum, Identity.Minimum);
}

In [0]:
public interface BoundedCashflows : IScope<CashflowIdentity, ICashflowStorage>
{
    public IEnumerable<(string Layer, double[] Values)> LayerValues => GetStorage().GetCashflow<BoundedCashflow>(Identity)
        .Select((cd,i) => ("Bounded-" + cd.Shape + i.ToString(), GetScope<ComputeBoundedCashflows>(cd).Values));     
}

# Reference Cashflow

In [0]:
public interface ReferencedCashflows : IScope<(CashflowIdentity Id, string OriginalAmountType), ICashflowStorage>{
   static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<ReferencedCashflows>(s => s
            .WithApplicability<ParametrisedCashflows>(x => x.Identity.OriginalAmountType == x.Identity.Id.AmountType));
        
    public double[] Values => GetScope<ReferenceCashflows>(Identity.Id).LayerValues.Concat(
        GetScope<ParametrisedCashflows>(Identity).LayerValues)
        .Select(x => x.Values).AggregateDoubleArray();
}

public interface ParametrisedCashflows : ReferencedCashflows{
    public IEnumerable<(string Layer, double[] Values)> LayerValues => GetScope<BasicCashflows>(Identity.Id).LayerValues.Concat(
        GetScope<BoundedCashflows>(Identity.Id).LayerValues);
    double[] ReferencedCashflows.Values => LayerValues.Select(x => x.Values).AggregateDoubleArray();
}

//Use functions where the reference can be provided externally
public interface WeightedReference : IScope<(CashflowIdentity Id, string ReferenceAmountType), ICashflowStorage>{
       static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<WeightedReference>(s => s
            .WithApplicability<WeightedSelfReference>(x => x.Identity.ReferenceAmountType == x.Identity.Id.AmountType)
            //.WithApplicability<ExternalReference>(x => x.GetStorage().IsExternalReference(x.Identity.ReferenceAmountType) ) //wip
            );

    protected double[] referencedValues => GetScope<ReferencedCashflows>((Identity.Id with {AmountType = Identity.ReferenceAmountType}, Identity.Id.AmountType)).Values;
    // double[] referencedValues => GetStorage().GetPillar() switch { //HERE the class method is used!
    //     "Solvency" => new []{1d,1d,1d,1d,1d},
    //     _ => new []{1000d,1000d,1000d,1000d,1000d},
    // };
    protected double[] weights => GetStorage().GetCashflow<ReferencedCashflow>(Identity.Id).Single(cd => cd.ReferenceAmountType == Identity.ReferenceAmountType).Values;
    
    double[] Values => weights.Zip(referencedValues, (weight, referencedValue) => weight * referencedValue).ToArray();
}
public interface ExternalReference : WeightedReference{
    double[] WeightedReference.referencedValues => GetStorage().GetValues(Identity.ReferenceAmountType);
}

public interface WeightedSelfReference : WeightedReference{
    double[] WeightedReference.Values => weights.Zip(referencedValues, (weight, referencedValue) => ( weight - 1d )  * referencedValue).ToArray();
}

public interface ReferenceCashflows : IScope<CashflowIdentity, ICashflowStorage>
{
    public IEnumerable<(string Layer, double[] Values)> LayerValues => GetStorage().GetCashflow<ReferencedCashflow>(Identity)
        .Select( (cd,i) => ("Reference" + cd.Shape + i.ToString(), 
                            GetScope<WeightedReference>((Identity, cd.ReferenceAmountType)).Values));
}

# Compute all Cashflows (no pillar storage - debug -)

In [0]:
public interface Compute : IScope<CashflowIdentity, CashflowStorage>
{
    public FpaVariable[] FpaVariables => (Identity.Type.Name switch {
        nameof(BasicCashflow) => GetScope<BasicCashflows>(Identity).LayerValues,
        nameof(BoundedCashflow) => GetScope<BoundedCashflows>(Identity).LayerValues,
        nameof(ReferencedCashflow) => GetScope<ReferenceCashflows>(Identity).LayerValues,
        _ => Enumerable.Empty<(string Layer, double[] Values)>(),
    }).Select(v => new FpaVariable() {
            Layer = v.Layer,
            Values = v.Values,
            DataNode = Identity.DataNode,
            AmountType = Identity.AmountType,
            AccidentYear = Identity.AccidentYear,
            EstimateType = EstimateTypes.BE,       //hardcodedValue
            Novelty = Novelties.C,                 //hardcodedValue
            AocType = AocTypes.CL                  //hardcodedValue
        }).ToArray();
}

# Solvency Scopes

In [0]:
public interface ComputeSolvency : IScope<CashflowIdentity, SolvencyStorage>
{
    public FpaVariable[] FpaVariables => (Identity.Type.Name switch {
        nameof(BasicCashflow) => GetScope<BasicCashflows>(Identity).LayerValues,
        nameof(BoundedCashflow) => GetScope<BoundedCashflows>(Identity).LayerValues,
        nameof(ReferencedCashflow) => GetScope<ReferenceCashflows>(Identity).LayerValues,
        _ => Enumerable.Empty<(string Layer, double[] Values)>(),
    }).Select(v => new FpaVariable() {
            Layer = v.Layer,
            Values = v.Values,
            DataNode = Identity.DataNode,
            AmountType = Identity.AmountType,
            AccidentYear = Identity.AccidentYear,
            EstimateType = EstimateTypes.BE,       //hardcodedValue
            Novelty = Novelties.C,                 //hardcodedValue
            AocType = AocTypes.CL                  //hardcodedValue
        }).ToArray();
}